In [1]:
#Install elastic search
!pip install elasticsearch


In [2]:
!pip install -q pandas openai

In [3]:
# To search documents 

#Import Elasticsearch and helpers from  elasticsearch

from elasticsearch import Elasticsearch,helpers
import json

es = Elasticsearch(['http://localhost:9200'])
es.ping()   #connection testing
res = es.search(index="github_issues", body={"query": {"match_all": {}}})

# Extract the total hit count
print("Got %d Hits:" % res['hits']['total']['value'])

# Extract hits from the response
hits = res.get('hits', {})

# Pretty print the hits
print(json.dumps(hits, indent=2))

Got 2373 Hits:
{
  "total": {
    "value": 2373,
    "relation": "eq"
  },
  "max_score": 1.0,
  "hits": [
    {
      "_index": "github_issues",
      "_id": "HdZXA48B8oHyUkvbPJJU",
      "_score": 1.0,
      "_source": {
        "_type": "issue",
        "_repo": "elasticsearch",
        "_issueNumber": "105928",
        "_title": "[ML] Anomaly detection job reset can get stuck with no way to unblock",
        "_createdAt": "2024-03-04T18:15:53Z",
        "_closedAt": "2024-03-11T15:40:24Z",
        "_state": "closed",
        "_body": "When an anomaly detection job is reset we start a local (non-persistent) task to do the resetting operations.\r\n\r\nIt is possible for this task to disappear before it completes. One well-known way this can happen is if the node it is running on dies while it is running.\r\n\r\nLooking at the code, you'd think that a way to unblock the reset would be to call the reset endpoint again for the same job. It appears that `resetIfJobIsStillBlockedOnReset` 

In [4]:
hits['hits'][5]['_source']  # Choose an Issue from Issues in repository

{'_type': 'issue',
 '_repo': 'elasticsearch',
 '_issueNumber': '105923',
 '_title': 'BWC test failures for health/10_basic/cluster health basic test',
 '_createdAt': '2024-03-04T16:42:54Z',
 '_closedAt': '2024-03-04T18:34:42Z',
 '_state': 'closed',
 '_body': '### CI Link\r\n\r\nhttps://gradle-enterprise.elastic.co/s/j2zhn5js47wcy\r\n\r\n### Repro line\r\n\r\n./gradlew \':qa:mixed-cluster:v8.5.3#mixedClusterTest\' -Dtests.class="org.elasticsearch.backwards.MixedClusterClientYamlTestSuiteIT" -Dtests.method="test {p0=health/10_basic/cluster health basic test}" -Dtests.seed=72A1B4518E0C3ECD -Dtests.bwc=true -Dtests.locale=es-UY -Dtests.timezone=America/Blanc-Sablon -Druntime.java=21\r\n\r\n### Does it reproduce?\r\n\r\nNo\r\n\r\n### Applicable branches\r\n\r\n8.13, main\r\n\r\n### Failure history\r\n\r\n_No response_\r\n\r\n### Failure excerpt\r\n\r\nThis is likely due to https://github.com/elastic/elasticsearch/pull/105789\r\n\r\n```\r\njava.lang.AssertionError: Failure at [health/10_basi

In [5]:
import json

response = es.search(
  index="github_issues",
  knn={
      "field": "GitHub_Issue_vector",  
      "query_vector": hits['hits'][5]['_source']["GitHub_Issue_vector"], # Finding Issues that are similar to that Issue
      "k": 50,
      "num_candidates": 10000
    }
)
responses=[]
for response in response["hits"]["hits"]:
    response_obj={
        "_score":response["_score"],
        "_type":response["_source"]["_type"],
        "_repo":response["_source"]["_repo"],
        "_issueNumber":response["_source"]["_issueNumber"],
        "_title":response["_source"]["_title"],
        "_createdAt":response["_source"]["_createdAt"],
        "_closedAt":response["_source"]["_closedAt"],
         "_body":response["_source"]["_body"],
    }
    responses.append(response_obj)


In [6]:
# We get Issues that are similar to the Issue chosen and sorted based on score
# First result is the most relevant and has the score 1.0 
for response in responses:
    print(response)
    print()

{'_score': 1.0, '_type': 'issue', '_repo': 'elasticsearch', '_issueNumber': '105923', '_title': 'BWC test failures for health/10_basic/cluster health basic test', '_createdAt': '2024-03-04T16:42:54Z', '_closedAt': '2024-03-04T18:34:42Z', '_body': '### CI Link\r\n\r\nhttps://gradle-enterprise.elastic.co/s/j2zhn5js47wcy\r\n\r\n### Repro line\r\n\r\n./gradlew \':qa:mixed-cluster:v8.5.3#mixedClusterTest\' -Dtests.class="org.elasticsearch.backwards.MixedClusterClientYamlTestSuiteIT" -Dtests.method="test {p0=health/10_basic/cluster health basic test}" -Dtests.seed=72A1B4518E0C3ECD -Dtests.bwc=true -Dtests.locale=es-UY -Dtests.timezone=America/Blanc-Sablon -Druntime.java=21\r\n\r\n### Does it reproduce?\r\n\r\nNo\r\n\r\n### Applicable branches\r\n\r\n8.13, main\r\n\r\n### Failure history\r\n\r\n_No response_\r\n\r\n### Failure excerpt\r\n\r\nThis is likely due to https://github.com/elastic/elasticsearch/pull/105789\r\n\r\n```\r\njava.lang.AssertionError: Failure at [health/10_basic:7]: expect